In [250]:
import pymysql
from datetime import datetime
import pandas as pd

In [251]:
clave = "root"

In [252]:
conexion = pymysql.connect(
    host = "localhost",
    database = "henry",
    user = "root",
    password = clave
)

In [253]:
cursor = conexion.cursor()

Armar un dataframe. Primero traigo las columnas y despues las filas.

In [254]:
cursor.execute("show columns from carrera")
columnas = cursor.fetchall()
nombre_col = [columna[0] for columna in columnas]
nombre_col

['idCarrera', 'nombre']

In [255]:
cursor.execute("select * from carrera")
alumnos = cursor.fetchall()
df = pd.DataFrame(alumnos, columns = nombre_col)
df

,idCarrera,nombre
0,1,Full Stack Developer
1,2,Data Science


In [256]:
df["idCarrera"].sum() #Utilizo pandas con el df recientemente creado

3

Probando con pandas y PyMySQL.

In [257]:
sql = "SELECT * FROM alumno"
df = pd.read_sql(sql, conexion)
conexion.close()
print(df.head())

   idAlumno cedulaIdentidad   nombre    apellido fechaNacimiento fechaIngreso  \
0         1       274352477  Candice       Rojas      2000-12-10   2020-01-21   
1         2       313262855  Beverly     Gardner      2006-10-03   2019-12-04   
2         3       335199030   Carlos       Frank      2006-07-11   2019-12-13   
3         4       171722128   Callum   Velazquez      2005-11-18   2019-12-17   
4         5       14374907K    Kelly  Strickland      2005-11-26   2019-12-07   

   idCohorte  edad  
0       1235    22  
1       1235    16  
2       1235    17  
3       1235    17  
4       1235    17  


C:\Users\Tamara Campos\AppData\Local\Temp\ipykernel_3680\3164974825.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql, conexion)


Seleccionar sólo los alumnos que se ingresaron después de cierta fecha.

In [258]:
"""
#select * 
from alumno
where fechaIngreso > '2022-02-01';
"""

fecha_limite = pd.to_datetime("2022-02-13")
alumnos_postfecha = df[df["fechaIngreso"] > fecha_limite]
alumnos_postfecha

C:\Users\Tamara Campos\AppData\Local\Temp\ipykernel_3680\1199404528.py:8: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  alumnos_postfecha = df[df["fechaIngreso"] > fecha_limite]


,idAlumno,cedulaIdentidad,nombre,apellido,fechaNacimiento,fechaIngreso,idCohorte,edad
121,122,57652322,Edan,Ferguson,2002-05-29,2022-02-19,1241,21
123,124,383749255,Jason,Harmon,1990-08-17,2022-02-27,1241,33
145,146,10235723,Grace,Crane,2007-11-27,2022-02-22,1242,15
146,147,112593543,Rhoda,Golden,2001-04-27,2022-02-23,1242,22
147,148,369224867,McKenzie,Silva,2008-12-28,2022-02-24,1242,14
152,153,247172483,Zelda,Curtis,2009-10-10,2022-02-21,1242,13
154,155,368514128,Duncan,Vasquez,1992-09-18,2022-02-25,1242,31
157,158,319157859,Azalia,Weiss,2000-09-20,2022-02-25,1242,22
160,161,41944781,Stephanie,Hurst,1986-11-23,2022-02-14,1243,36
166,167,448157210,Kiayada,Oneil,2009-03-26,2022-02-25,1243,14


Ordenar los alumnos por fecha de nacimiento de forma descendente.

In [259]:
"""
select * 
from alumno
order by fechaNacimiento desc;
"""

alumnos_ordenados = df.sort_values(by = "fechaNacimiento", ascending = False)
alumnos_ordenados

,idAlumno,cedulaIdentidad,nombre,apellido,fechaNacimiento,fechaIngreso,idCohorte,edad
70,71,388551046,Fitzgerald,Frederick,2009-12-29,2021-08-02,1238,13
138,139,46150244,Darius,Schmidt,2009-12-19,2022-01-16,1241,13
83,84,408178630,Abbot,Moss,2009-12-15,2021-10-24,1239,13
179,180,277347865,William,Long,2009-12-06,2022-02-09,1243,13
6,7,398182472,Aimee,Cline,2009-11-19,2020-01-26,1235,13
...,...,...,...,...,...,...,...,...
34,35,381918319,Margaret,Bentley,1981-07-25,2020-02-17,1236,42
56,57,402475935,Ian,Stephenson,1981-06-19,2021-03-30,1237,42
140,141,468707764,Hedda,Wells,1980-11-30,2022-01-18,1242,42
57,58,474328023,Cooper,Duncan,1980-09-18,2021-01-07,1237,43


Agregar una columna al DataFrame que contenga la edad de cada alumno en años. Encontrar el promedio de edad de los alumnos.

In [260]:
"""
alter table alumno add column edad int;
/*Con esto logro autorizar la actualización de la tabla*/
SET SQL_SAFE_UPDATES = 0;
update alumno set edad = datediff(curdate(), fechaNacimiento) / 365;
/*Luego de realizar el cambio vuelva a colocar 1 ya que puede generar grandes perdidas*/
SET SQL_SAFE_UPDATES = 1;
select * from alumno;

select avg(edad) as promedio
from alumno;
"""

hoy = datetime.today()
df = df.drop(126)
df["edad"] = (hoy - pd.to_datetime(df["fechaNacimiento"])).astype('<m8[Y]')
promedio_edad = df["edad"].mean()
round(promedio_edad, 2)

22.14

Encontrar la cantidad de alumnos por cohorte.

In [261]:
"""
select count(*) as cantidad, IdCohorte
from alumno
group by IdCohorte;
"""

alumnos_cohorte = df["idCohorte"].value_counts()
alumnos_cohorte

1235    20
1236    20
1237    20
1238    20
1239    20
1240    20
1242    20
1243    20
1241    19
Name: idCohorte, dtype: int64

Seleccionar sólo los alumnos cuya cédula de identidad comienza con un cierto prefijo.

In [262]:
"""
select *
from alumno
where cedulaIdentidad like "20%";
"""

cedula_pre = df["cedulaIdentidad"].str.startswith("277")
alumnos_pre = df[cedula_pre]
alumnos_pre

,idAlumno,cedulaIdentidad,nombre,apellido,fechaNacimiento,fechaIngreso,idCohorte,edad
44,45,277992078,Gray,Willis,1993-02-08,2021-03-26,1237,30.0
179,180,277347865,William,Long,2009-12-06,2022-02-09,1243,13.0


Encontrar la fecha de ingreso más temprana y más tardía de los alumnos.

In [263]:
""" 
select min(FechaIngreso) as fecha_minima, max(FechaIngreso) as fecha_maxima
from alumno;
"""

fecha_min_ing = df["fechaIngreso"].min()
fecha_max_ing = df["fechaIngreso"].max()
print(fecha_min_ing, fecha_max_ing)

2019-12-04 2022-02-27


Seleccionar sólo los alumnos que tengan una edad entre dos valores dados.

In [264]:
"""
select *
from alumno
where edad < 20 and edad > 18;
"""

edad_entre = df[(df["edad"] > 23) & (df["edad"] < 25)]
edad_entre

,idAlumno,cedulaIdentidad,nombre,apellido,fechaNacimiento,fechaIngreso,idCohorte,edad
135,136,45903699,Shelby,Kinney,1998-04-16,2022-01-04,1241,24.0
153,154,213189034,Bethany,Hoover,1998-12-24,2022-01-02,1242,24.0
